In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
import warnings
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss
warnings.filterwarnings('ignore')

#### 用convert_dtypes()读取数据

In [2]:
train = pd.read_csv('./input/train.csv').convert_dtypes(convert_string=False)
testa = pd.read_csv('./input/testA.csv').convert_dtypes(convert_string=False)

In [3]:
# train.dtypes Int

#### issueDate

#### 这里对了吗？？？datetime? string?

In [4]:
#转化成时间格式，这里暂时不新增，感觉意义不明确，但是查了确实train的最小日期是2007-6-1
for data in [train, testa]:
    data['issueDate'] = pd.to_datetime(data['issueDate'],format='%Y-%m-%d')
    startdate = datetime.datetime.strptime('2007-06-01', '%Y-%m-%d')
    # 构造时间特征
    data['issueDateDT'] = data['issueDate'].apply(lambda x: x-startdate).dt.days

In [5]:
# train.dtypes   # issueDate datetime64[ns]

In [ ]:
pd.set_option('display.max_columns', None)
train.describe(include="all")

In [7]:
train.dtypes

id                             Int64
loanAmnt                       Int64
term                           Int64
interestRate                 float64
installment                  float64
grade                         object
subGrade                      object
employmentTitle                Int64
employmentLength              object
homeOwnership                  Int64
annualIncome                 float64
verificationStatus             Int64
issueDate             datetime64[ns]
isDefault                      Int64
purpose                        Int64
postCode                       Int64
regionCode                     Int64
dti                          float64
delinquency_2years             Int64
ficoRangeLow                   Int64
ficoRangeHigh                  Int64
openAcc                        Int64
pubRec                         Int64
pubRecBankruptcies             Int64
revolBal                       Int64
revolUtil                    float64
totalAcc                       Int64
i

#### employmentLength的 years删掉，10+ <1转

In [8]:
def employmentLength_to_int(s):
    if pd.isnull(s):
        return s
    else:
        return np.int8(s.split()[0])
for data in [train, testa]:
    data['employmentLength'].replace(to_replace='10+ years', value='10 years', inplace=True)
    data['employmentLength'].replace('< 1 year', '0 years', inplace=True)
    data['employmentLength'] = data['employmentLength'].apply(employmentLength_to_int)

In [9]:
train['employmentLength'].value_counts(dropna=False).sort_index()

0.0      64237
1.0      52489
2.0      72358
3.0      64152
4.0      47985
5.0      50102
6.0      37254
7.0      35407
8.0      36192
9.0      30272
10.0    262753
NaN      46799
Name: employmentLength, dtype: int64

#### 这里还是float要转吗，一会再说吧

In [10]:
# train['employmentLength'].astype('Int') # 没这种写法

In [11]:
# test[]

#### earliesCreditLine 这个要怎么弄

In [ ]:
def employmentLength_to_int(s):
    if pd.isnull(s):
        return s
    else:
        return np.int8(s.split()[0])
for data in [train, testa]:
    data['employmentLength'].replace(to_replace='10+ years', value='10 years', inplace=True)
    data['employmentLength'].replace('< 1 year', '0 years', inplace=True)
    data['employmentLength'] = data['employmentLength'].apply(employmentLength_to_int)

In [12]:
pd.set_option('display.max_rows', None)

In [13]:
train['earliesCreditLine'].sample(50)

340435    Aug-2003
659107    Apr-1997
689270    Nov-1998
578057    Nov-1996
223358    Oct-1980
783655    Feb-2006
531124    Nov-2001
304445    Nov-2001
517296    Apr-1993
262596    May-2006
791150    Sep-2004
608436    May-2002
703051    Jun-1999
661280    Jan-1997
333799    Jan-2000
289534    Nov-1986
591978    Jun-1998
275716    Oct-1972
429897    Nov-1997
639743    Dec-2000
433086    May-1997
143156    Jan-1995
305264    Feb-1997
347630    May-1998
543705    Sep-1987
542810    May-2000
603384    Mar-2002
25103     Feb-2003
370986    Apr-1988
778433    Oct-1998
122248    Dec-2000
584087    May-1997
559761    Nov-1998
642613    Apr-1984
743408    Oct-1997
229075    Jan-2003
307084    Nov-1998
418885    Feb-2000
793732    Nov-2004
728582    Nov-2006
627908    Jul-2011
403639    Nov-2010
494014    Jun-1997
667226    Mar-1995
515933    Aug-1993
251536    Mar-2005
91696     Sep-2010
147976    Jul-1997
333331    Jun-2006
57511     Dec-1998
Name: earliesCreditLine, dtype: object

In [14]:
for data in [train, testa]:
    data['earliesCreditLine'] = data['earliesCreditLine'].apply(lambda s: int(s[-4:]))

#### 压缩一下内存

In [15]:
## Simple "Memory profilers" to see memory usage
def get_memory_usage():
    return np.round(psutil.Process(os.getpid()).memory_info()[0]/2.**30, 2) 
        
def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

In [16]:
## Memory Reducer
# :df pandas dataframe to reduce size             # type: pd.DataFrame()
# :verbose                                        # type: bool
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                       df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [17]:
# print("{:>20}: {:>8}".format('Original grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))

这没写呢

'earliesCreditLine'不会处理先drop掉

#### 数据类型转换

In [18]:
for data in [train, testa]:
    data['employmentTitle'] = data['employmentTitle'].astype('int32')

ValueError: cannot convert to 'int32'-dtype NumPy array with missing values. Specify an appropriate 'na_value' for this dtype.

In [ ]:
train['employmentTitle'] = train['employmentTitle'].astype('int32')

In [ ]:
cat_fea = ['grade', 'subGrade', 'employmentTitle', 'verificationStatus', \
          'purpose', 'postCode', 'regionCode','applicationType','initialListStatus', 'title', 'policyCode']

In [ ]:
cols = ['id', 'loanAmnt', 'term', 'interestRate', 'installment', 'grade',
       'subGrade', 'employmentTitle', 'employmentLength', 'homeOwnership',
       'annualIncome', 'verificationStatus', 'issueDate', 'isDefault',
       'purpose', 'postCode', 'regionCode', 'dti', 'detlinquency_2years',
       'ficoRangeLow', 'ficoRangeHigh', 'openAcc', 'pubRec',
       'pubRecBankruptcies', 'revolBal', 'revolUtil', 'totalAcc',
       'initialListStatus', 'applicationType', 'earliesCreditLine', 'title',
       'policyCode', 'n0', 'n1', 'n2', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9',
       'n10', 'n11', 'n12', 'n13', 'n14']

'issueDate', 'isDefault', 'earliesCreditLine'

In [ ]:
num_fea = [a for a in cols if a not in cat_fea]
num_fea

In [ ]:
## 定义了一个统计函数，方便后续信息统计
def Sta_inf(data):
    print('_min',np.min(data))
    print('_max:',np.max(data))
    print('_mean',np.mean(data))
    print('_ptp',np.ptp(data))
    print('_std',np.std(data))
    print('_var',np.var(data))